<a href="https://colab.research.google.com/github/sayakpaul/Grocery-Product-Detection/blob/main/Colabs/GroceryDataset_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook generates recall and precision scores (for 0.5IOU) on the test grocery dataset with the model trained in `Colabs/GroceryDataset_Model_Training.ipynb` notebook. 

## Initial setup

In [ ]:
# Install TFOD API (TF 1)
%tensorflow_version 1.x
import tensorflow as tf 
print(tf.__version__)

!git clone https://github.com/tensorflow/models.git

% cd models/research
!pip install --upgrade pip
# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf1/setup.py .
!python -m pip install --use-feature=2020-resolver .

## Gather test record file, trained model graph and `.pbtxt`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
!cp -r /content/drive/MyDrive/product-detection/test.record .
!cp -r /content/drive/MyDrive/product-detection/label_map.pbtxt .
!cp -r /content/drive/MyDrive/product-detection/fp32/frozen_inference_graph.pb .

## Gather detections

In [5]:
# Infer detections on a TFRecord of TFExamples given an inference graph.
!python object_detection/inference/infer_detections.py \
    --input_tfrecord_paths=test.record \
    --output_tfrecord_path=detections.tfrecord \
    --inference_graph=frozen_inference_graph.pb

2021-01-17 05:03:42.074377: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-01-17 05:03:42.129133: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-01-17 05:03:42.129711: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:00:04.0
2021-01-17 05:03:42.129973: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-01-17 05:03:42.339943: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-01-17 05:03:42.476808: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2021-

## Generate metrics

In [6]:
!git clone https://github.com/svpino/tf_object_detection_cm

Cloning into 'tf_object_detection_cm'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 35 (delta 0), reused 3 (delta 0), pack-reused 31
Unpacking objects: 100% (35/35), done.


In [7]:
!python tf_object_detection_cm/confusion_matrix.py \
    --detections_record=detections.tfrecord \
    --label_map=label_map.pbtxt \
    --output_path=confusion_matrix.csv


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
W0117 05:06:05.042840 139941604333440 deprecation.py:323] From tf_object_detection_cm/confusion_matrix.py:38: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Processed 71 images

Confusion Matrix:
[[1.964e+03 0.000e+00 8.000e+00 0.000e+00 2.000e+00 1.000e+00 1.300e+01
  7.000e+00 3.000e+00 0.000e+00 1.000e+00 2.400e+01]
 [0.000e+00 4.700e+01 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [2.900e+01 0.000e+00 2.200e+02 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 3.000e+00]
 [6.000e+00 0.000e+00 0.000e+00 1.000e+01 0.000e+00 0.000e+00 0.000e+00
  3.000e+00 0.000e+00 0.000e+00 0.000e+00 2.000e+00]
 [6.000e+00 0.000e+00 0.000e+00 0.000e+00 8.900e+01 0.000e+00 0.